# Sheep in a Meadow

---

This notebook is part of a cadCAD meetup. The associated slides are available [here](https://docs.google.com/presentation/d/1JrsrD-ZEmlOYFujKNsCerULEzFq32gOrZ8wizsbhC_4/edit?usp=sharing).

The example presented is inspired by the [cadCAD Complete Foundations Bootcamp](https://www.cadcad.education/course/bootcamp) from [cadCAD Edu](https://www.cadcad.education/). Highly recommended!

---

# Table of Contents

1. [System Requirements](#1.-System-Requirements)
  * [Requirements Anaylsis](#Requirements-Analysis)
  * [Visual System Mappings](#Visual-System-Mappings)
  * [Mathematical Specification](#Mathematical-Specification)


2. [System Design](#2.-System-Design)
  * [Differential Specification](#Differential-Specification)
  * [Modelling](#Modelling)
  * [Simulation](#Simulation)


3. [System Validation](#3.-System-Validation)
    * [Model Improvement (1)](#Model-Improvement-(1))
      * [Mathematical Specification Update (1)](#Mathematical-Specification-Updat-(1))
      * [Differential Specification Update (1)](#Differential-Specification-Update-(1))
      * [Modelling & Simulation Update (1)](#Modelling-&-Simulation-Update-(1))
    * [Model Improvement (2)](#Model-Improvement-(2))
      * [Mathematical Specification Update (2)](#Mathematical-Specification-Update-(2))
      * [Differential Specification Update (2)](#Differential-Specification-Update-(2))
      * [Modelling & Simulation Update (2)](#Modelling-&-Simulation-Update-(2))


---

---

# 1. System Requirements

<center><img src="images/requirements.png" alt="Requirements" width="70%"/>

## Requirements Analysis

**Goal:** Simulate animal food consumption

**Scope:** Learn the fundamentals of cadCAD engineering very quickly

**Question:** How long will our model ecosystem be self-sustaining?

**Assumptions:**
- The population increases over time
- The food supply decreases over time
- There is some relationship between the population and food supply

## Visual System Mappings

#### Entity Relationship Diagram

<center><img src="images/entity-relationship-diagram.png" alt="Entity Relationship Diagram" width="60%"/>

#### Stock & Flow Diagram

<center><img src="images/stock-and-flow-diagram.png" alt="Stock & Flow Diagram" width="70%"/>

## Mathematical Specification

<center><img src="images/equations.png" alt="Food Consumption Equations" width="50%"/>

[Go to Mathematical Specification Update (1)](#Mathematical-Specification-Update-(1))

---

---

# 2. System Design

<center><img src="images/design.png" alt="Design" width="70%"/>

## Differential Specification

<center><img src="images/differential-specification-1.png" alt="Differential Specification" width="90%"/>

[Go to Differential Specification Update (1)](#Differential-Specification-Update-(1))

## Modelling

In [ ]:
##############
# 0. IMPORTS #
##############

# Standard libraries
import math
from collections import Counter

# Analysis and plotting modules
import pandas as pd
import plotly.express as px

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD stores simulation configuration in the configs list
from cadCAD import configs

In [ ]:
######################
# 1. STATE VARIABLES #
######################

initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}


########################
# 2. SYSTEM PARAMETERS #
########################

system_params = {
    'reproduction_rate': [0.01], # sheep per month
    'consumption_rate': [0.01], # tons of grass per month
}


#######################
# 3. POLICY FUNCTIONS #
#######################

def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}


#############################
# 4. STATE UPDATE FUNCTIONS #
#############################

def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food'] 
    return 'food', max(food, 0)


##################################
# 5. PARTIAL STATE UPDATE BLOCKS #
##################################

# Updates run in parallel
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'consumption': p_consumption
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

## Simulation

In [ ]:
####################
# 6. CONFIGURATION #
####################

# the config_sim function creates an object that informs cadCAD about the scope of the simulation
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation
    "T": range(600), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

# Clear any prior configs
del configs[:]

# each cacCAD experiment has configuration options which
# we will pass to the append_configs function to create our simulation configuration
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)


################
# 7. EXECUTION #
################

exec_context = ExecutionContext() # decides how cadCAD should run a simulation (default: single or multi threaded)
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute() # execute the simulation


####################################
# 8. SIMULATION OUTPUT PREPARATION #
####################################

simulation_result = pd.DataFrame(raw_result)
#simulation_result.head()


##########################
# 9. SIMULATION ANALYSIS #
##########################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['population','food']
)

---

---

# 3. System Validation

<center><img src="images/validation.png" alt="Validation" width="70%"/>

## Model Improvement (1)

### Mathematical Specification Update (1)

<center><img src="images/equations-update-1.png" alt="Updated Food Consumption Equations" width="50%"/>

* [Go to Mathematical Specification](#Mathematical-Specification)
* [Go to Mathematical Specification Update (2)](#Mathematical-Specification-Update-(2))

### Differential Specification Update (1)

<center><img src="images/differential-specification-2.png" alt="Differential Specification" width="90%"/>

* [Go to Differential Specification](#Differential-Specification)
* [Go to Differential Specification Update (2)](#Differential-Specification-Update-(2))

### Modelling & Simulation Update (1)

In [ ]:
######################
# 1. STATE VARIABLES #
######################

initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}


########################
# 2. SYSTEM PARAMETERS #
########################

system_params = {
    'reproduction_rate': [0.01], # sheep per month
    'consumption_rate': [0.01], # tons of grass per month
    'growth_rate': [15.0], # tons of grass per month (new param)
}


#######################
# 3. POLICY FUNCTIONS #
#######################

def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}

# new policy function
def p_growth(params, substep, state_history, previous_state):
    delta_food = params['growth_rate']
    return {'delta_food': delta_food}


#############################
# 4. STATE UPDATE FUNCTIONS #
#############################

def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food'] 
    return 'food', max(food, 0)


##################################
# 5. PARTIAL STATE UPDATE BLOCKS #
##################################

# Updates run in parallel
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'consumption': p_consumption,
            'growth': p_growth # new policy function
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [ ]:
####################
# 6. CONFIGURATION #
####################

# the config_sim function creates an object that informs cadCAD about the scope of the simulation
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation
    "T": range(600), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

# Clear any prior configs
del configs[:]

# each cacCAD experiment has configuration options which
# we will pass to the append_configs function to create our simulation configuration
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)


################
# 7. EXECUTION #
################

exec_context = ExecutionContext() # decides how cadCAD should run a simulation (default: single or multi threaded)
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute() # execute the simulation


####################################
# 8. SIMULATION OUTPUT PREPARATION #
####################################

simulation_result = pd.DataFrame(raw_result)
#simulation_result.head()


##########################
# 9. SIMULATION ANALYSIS #
##########################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['population','food']
)

## Model Improvement (2)

### Mathematical Specification Update (2)

<center><img src="images/equations-update-2.png" alt="Updated Food Consumption Equations" width="50%"/>

[Go to Mathematical Specification Update (1)](#Mathematical-Specification-Update-(1))

### Differential Specification Update (2)

<center><img src="images/differential-specification-3.png" alt="Differential Specification" width="90%"/>

[Go to Differential Specification Update (1)](#Differential-Specification-Update-(1))

### Modelling & Simulation Update (2)

In [ ]:
######################
# 1. STATE VARIABLES #
######################

initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}


########################
# 2. SYSTEM PARAMETERS #
########################

system_params = {
    'reproduction_rate': [0.01], # sheep per month
    'consumption_rate': [0.01], # tons of grass per month
    'growth_rate': [15.0], # tons of grass per month
    'death_rate': [0.01], # sheep per month (new param)
}


#######################
# 3. POLICY FUNCTIONS #
#######################

def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}

def p_growth(params, substep, state_history, previous_state):
    delta_food = params['growth_rate']
    return {'delta_food': delta_food}

# new policy function
def p_death(params, substep, state_history, previous_state):
    population_death = params['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}


#############################
# 4. STATE UPDATE FUNCTIONS #
#############################

def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food'] 
    return 'food', max(food, 0)


##################################
# 5. PARTIAL STATE UPDATE BLOCKS #
##################################

# Updates run in parallel
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'consumption': p_consumption,
            'growth': p_growth,
            'death': p_death, # new policy function
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [ ]:
####################
# 6. CONFIGURATION #
####################

# the config_sim function creates an object that informs cadCAD about the scope of the simulation
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation
    "T": range(600), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

# Clear any prior configs
del configs[:]

# each cacCAD experiment has configuration options which
# we will pass to the append_configs function to create our simulation configuration
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)


################
# 7. EXECUTION #
################

exec_context = ExecutionContext() # decides how cadCAD should run a simulation (default: single or multi threaded)
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute() # execute the simulation


####################################
# 8. SIMULATION OUTPUT PREPARATION #
####################################

simulation_result = pd.DataFrame(raw_result)
#simulation_result.head()


##########################
# 9. SIMULATION ANALYSIS #
##########################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['population','food']
)

In [ ]:
df = simulation_result.copy()

fig = px.scatter(
    df,
    x='population',
    y='food',
    color='timestep'
)
fig.show()